In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum
import pyspark.sql
import pandas as pd
import numpy as np

In [4]:
spark = SparkSession.builder.getOrCreate()

In [10]:
train = spark.read.csv('../data/input/train.csv', header=True)
test = spark.read.csv('../data/input/test.csv', header=True)

In [7]:
train.printSchema()

root
 |-- date_time: string (nullable = true)
 |-- site_name: string (nullable = true)
 |-- posa_continent: string (nullable = true)
 |-- user_location_country: string (nullable = true)
 |-- user_location_region: string (nullable = true)
 |-- user_location_city: string (nullable = true)
 |-- orig_destination_distance: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- is_mobile: string (nullable = true)
 |-- is_package: string (nullable = true)
 |-- channel: string (nullable = true)
 |-- srch_ci: string (nullable = true)
 |-- srch_co: string (nullable = true)
 |-- srch_adults_cnt: string (nullable = true)
 |-- srch_children_cnt: string (nullable = true)
 |-- srch_rm_cnt: string (nullable = true)
 |-- srch_destination_id: string (nullable = true)
 |-- srch_destination_type_id: string (nullable = true)
 |-- is_booking: string (nullable = true)
 |-- cnt: string (nullable = true)
 |-- hotel_continent: string (nullable = true)
 |-- hotel_country: string (nullable = true)
 

How many samples does the train data include?

In [8]:
train.count()

37670293

Quite big! Around 38 millions!

In [12]:
train.describe().toPandas()

,summary,date_time,site_name,posa_continent,user_location_country,user_location_region,user_location_city,orig_destination_distance,user_id,is_mobile,...,srch_children_cnt,srch_rm_cnt,srch_destination_id,srch_destination_type_id,is_booking,cnt,hotel_continent,hotel_country,hotel_market,hotel_cluster
0,count,37670293,37670293,37670293,37670293,37670293,37670293,24145292,37670293,37670293,...,37670293,37670293,37670293,37670293,37670293,37670293,37670293,37670293,37670293,37670293
1,mean,None,9.795271329585889,2.6804730188851997,86.10880194109454,308.4060117610447,27753.044729330883,1970.0900267207285,604451.7531778422,0.1349265056154461,...,0.3321221579030458,1.1126628083301608,14441.090543760836,2.5822799148389954,0.07965674702875288,1.4833839227106622,3.1563047837190967,81.29685165974153,600.461883638654,49.80860501934509
2,stddev,None,11.9675435665128,0.7480393482506577,59.24310334783878,208.44374973856722,16782.553195680346,2232.4424303904275,350617.4620408585,0.34164505966916764,...,0.7314980986397146,0.45911549963856946,11066.302332627309,2.153018959399955,0.2707610600283715,1.2197755786558424,1.6231886782105662,56.171188062887815,511.73912727922396,28.915950805004293
3,min,2013-01-07 00:00:02,10,0,0,0,0,0.0056,0,0,...,0,0,0,0,0,1,0,0,0,0
4,max,2014-12-31 23:59:59,9,4,99,999,9998,9999.9984,999999,1,...,9,8,9999,9,1,99,6,99,999,99


In [11]:
test.describe().toPandas()

,summary,id,date_time,site_name,posa_continent,user_location_country,user_location_region,user_location_city,orig_destination_distance,user_id,...,srch_ci,srch_co,srch_adults_cnt,srch_children_cnt,srch_rm_cnt,srch_destination_id,srch_destination_type_id,hotel_continent,hotel_country,hotel_market
0,count,2528243,2528243,2528243,2528243,2528243,2528243,2528243,1680782,2528243,...,2528222,2528226,2528243,2528243,2528243,2528243,2528243,2528243,2528243,2528243
1,mean,1264121.0,None,9.70966556616591,2.697412788248598,85.51611929707705,317.0340683233376,27862.17806318459,1686.3979638184105,602645.9257690024,...,None,None,1.958126651591639,0.25074132510205704,1.117409995795499,16568.621168139296,2.941701806353266,3.022614914784694,81.25454277931354,646.4925847713214
2,stddev,729841.0326509465,None,12.271834283602896,0.7651287624994623,56.73879902333156,208.28338881273413,16687.877645606164,2197.361433047482,346965.11778565275,...,None,None,0.8980415286253776,0.6754566222733871,0.4615009555032746,12805.019853860984,2.282005929487735,1.6748793299399136,55.07528338943058,504.65298547297846
3,min,0,2015-01-08 00:00:35,0,0,0,0,0,0.0056,0,...,2015-01-04,2015-01-08,0,0,0,10,1,0,0,0
4,max,999999,2015-12-31 23:58:47,9,4,99,999,9999,9999.9851,999999,...,2161-10-00,2017-04-30,9,9,8,9999,9,6,99,999


In [ ]:
train_sp = train_sp.withColumn('is_booking', train_sp['is_booking'].astype("float"))

In [ ]:
train_sp[['is_booking']].describe().toPandas()

In [ ]:
train_sp.where(col('is_booking').isNull())

In [ ]:
train_sp.columns

In [ ]:
test = pd.read_csv('../data/input/test.csv')

In [ ]:
chunksize = 1 * 10 ** 6
train_reader = pd.read_csv('../data/input/train.csv', chunksize=chunksize)

In [ ]:
all_chunks = []
i = 0
for chunk in train_reader:
    all_chunks.append(chunk)
    print(i)
    i += 1

In [ ]:
train = pd.DataFrame()

In [ ]:
import time

In [ ]:
for i in range(5, 10):
    start_time = time.time()
    train = pd.concat(objs = [train, all_chunks[i]], axis = 0).reset_index(drop = True)
    end_time = time.time()
    print(i)
    print(end_time - start_time)

In [ ]:
train.head()